In [26]:
# dependencies
import hashlib
from numpy import nan as nan
import pandas as pd
from sklearn.model_selection import train_test_split
from math import ceil

### id tracking
| df | article_id | matchedsentence_id | officer_id |
|:--- | ---: | ---: | :---:|
| text_df | text_df.id|X|X|
| sen_df  | sen_df.article_id|sen_df.id|X|
| true_df | X |true_df.matchedsentence_id|true_df.id|

In [2]:
# support methods for notebook version
def get_unique_report(df):
    cols = list(df.columns)
    print('             distinct value count by col')
    print('=======================================================')
    for col in cols:
        pretty_print(col, len(df[col].unique()))

In [3]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--included")
    parser.add_argument("--true")
    parser.add_argument("--text")
    parser.add_argument("--output")
    return parser.parse_args()


def get_logging(logname):
        logging.basicConfig(level=logging.DEBUG,
                            format='%(asctime)s %(levelname)s %(message)s',
                            handlers=[logging.FileHandler(logname),
                            logging.StreamHandler()])


def open_gz(f):
    return pd.read_csv(f, compression='gzip')


def pretty_str(label, a, b=False, newline=False):
    if newline:
        if not b:
            return '{:50}{}{}'.format(label, a, '\n')
        else:
            return '{:50}{:10}{:10}{}'.format(label, a, b, '\n')
    if b:
        return '{:50}{:10}{:10}'.format(label, a, b)
    return '{:50}{}'.format(label, a)


def check_asserts(text_df, sen_df, true_df):
    assert text_df.shape == (29707, 12)
    assert sen_df.shape == (10470, 7)
    assert true_df.shape == (735, 3)
    assert all(text_df.columns == ['created_at', 'link', 'guid', 'source_id', \
                                   'updated_at', 'content', 'published_date', 'id', \
                                   'title', 'is_processed', 'author', 'url'])
    assert all(sen_df.columns == ['id', 'created_at', 'updated_at', 'article_id', 
                                   'extracted_keywords', 'text', 'title'])
    assert all(true_df.columns == ['id', 'matchedsentence_id', 'officer_id'])
    most = set(text_df.id.unique())
    mid = set(sen_df.id.unique())
    least = set(true_df.id.unique())
    assert len(least) < len(mid) < len(most)
    assert len(most.intersection(mid)) == 9891
    assert all(true_df.id == true_df.officer_id)   # what does it mean that this is true? will it always?
    pairs = set()
    for tup in true_df.itertuples():
        pairs.add((tup.id, tup.matchedsentence_id))
    assert len(pairs) == true_df.shape[0]
    articles = text_df.id.unique()
    matched = sen_df.article_id.unique()
    assert len(matched) < len(articles)
    assert len(articles) == 29707
    assert len(matched) == 4323
    for match in matched:
        assert match in articles
    matched_sen = sen_df.id.unique()
    true_match_sen = true_df.matchedsentence_id.unique()
    true_match_off = true_df.id.unique()
    assert len(true_match_sen) < len(matched_sen)
    assert len(matched_sen) == 10470
    assert len(true_match_sen) == 479
    for match in true_match_sen:
        assert match in matched_sen


def format_extracted_str(list_str):
    if list_str is not None:
        clean = list_str.replace('[', '').replace(']', '').replace('"', '').lower()
        if ',' in clean:
            return str({val for val in clean.split(',')})
        return str({clean})
    return None


def prep_dfs(text_df, sen_df, true_df):
    less_text = text_df.loc[:, ['id', 'source_id', 'author', 'title', 'content']]
    temp = less_text
    less_text = temp.rename(columns={'id':'article_id'})
    less_sen = sen_df.loc[:, ['id', 'article_id', 'text']]
    temp = less_sen
    less_sen = temp.rename(columns={'id':'matchedsentence_id'})
    less_sen['extracted_keywords'] = sen_df.extracted_keywords.apply(format_extracted_str)
    less_sen['kw_match'] = [1 for val in range(less_sen.shape[0])]
    less_true = true_df.loc[:, ['officer_id', 'matchedsentence_id']]
    less_true['relevant'] = [1 for val in range(less_true.shape[0])]
    return less_text, less_sen, less_true


def merge_dfs(less_text, less_sen, less_true):
    less_text = less_text.set_index('article_id')
    less_sen = less_sen.set_index('article_id')
    out = less_text.join(less_sen, on='article_id', how='outer').reset_index().set_index('matchedsentence_id')
    temp = less_true
    less_true = temp.set_index('matchedsentence_id')
    out = out.join(less_true, on='matchedsentence_id', how='outer')
    out = out.reset_index()
    out.kw_match.fillna(value=0, axis=0, inplace=True)
    out.relevant.fillna(value=0, axis=0, inplace=True)
    temp = out
    out['kw_match'] = temp.kw_match.astype(int)
    out['relevant'] = temp.relevant.astype(int)
    return out


# this method is called when relevant_articles and irrelevant_articles are not disjoint sets
# resolves conflict by upgrading all occurances of an article_id in relevant_articles to relevant
# ie. a sentence from an article is matched to an officer and appears in matchedsentence_officers data
#     a different sentence from same article is not matched and deemed irrelevant, appears in matchedsentence data
#     conflict occurs when datasets merged
def correct_relevant(df):
    copy = df.copy()
    relevant = copy.loc[copy.relevant == 1].article_id.unique().tolist()
    copy.loc[copy.article_id.isin(relevant), 'relevant'] = 1
    return copy


# This method builds the POSITIVE cases: keyword matched AND article relevant (per Rajiv)
def prep_pos_train_test(df, train_perc=0.80, test_perc=0.20):
    id_mask = (df.officer_id.notnull())
    possible = df.loc[id_mask].article_id.unique().tolist()
    train_list, test_list = train_test_split(possible, test_size=test_perc, train_size=train_perc, shuffle=True)
    assert set(train_list).isdisjoint(set(test_list))
    return train_list, test_list


# This method builds the NEGATIVE cases: keyword matched but not relevant
def prep_neg_train_test(df, pos_rate, curr_train_n, curr_test_n):
    assert 0 < pos_rate <= 0.5
    target_train = ceil(curr_train_n/pos_rate)
    target_test = ceil(curr_test_n/pos_rate)
    needed_train = target_train - curr_train_n
    needed_test = target_test - curr_test_n
    id_mask = (df.kw_match == 1) & (df.officer_id.isnull())
    possible = df.loc[id_mask].article_id.unique().tolist()
    assert needed_train + needed_test <= len(possible)
    train_list, test_list = train_test_split(possible, test_size=needed_test, train_size=needed_train, shuffle=True)
    assert set(train_list).isdisjoint(set(test_list))
    return train_list, test_list


def make_train_test_cols(df, pos_rate):
    copy = df.copy()
    # get pos/neg and train/test indice sets
    pos_train_idx, pos_test_idx = prep_pos_train_test(copy)
    neg_train_idx, neg_test_idx = prep_neg_train_test(copy, pos_rate, len(pos_train_idx), len(pos_test_idx))
    # train
    train_idx = pos_train_idx + neg_train_idx
    copy['train'] = [1 if val in train_idx else 0 for val in copy.article_id.values]
    # test
    test_idx = pos_test_idx + neg_test_idx
    copy['test'] = [1 if val in test_idx else 0 for val in copy.article_id.values]
    return copy[['article_id', 'matchedsentence_id', 'source_id', 'author', 'title', 'text', \
                'content', 'officer_id', 'extracted_keywords', 'kw_match', 'relevant', 'train', 'test']]


def make_train_test_df(df):
    full = df.loc[((df.train == 1) | (df.test == 1)), ['article_id', 'content', 'relevant', 'test']]
    full.drop_duplicates(subset='article_id', inplace=True)
    return full


# Since out.kw_match = out.relevant_count + out.irrelevant_count, and relevant can't be true without kw_match,
# (out.relevant_count) / (out.kw_match) should be the proportion of relevant samples given kw_match for col value
def make_report(df, col):
    kw_match_vc = df.loc[df.kw_match == 1][col].value_counts().to_dict()
    relevant_vc = df.loc[df.relevant == 1][col].value_counts().to_dict()
    irrelevant_match_vc = df.loc[(df.kw_match == 1) & (df.relevant != 1)][col].value_counts().to_dict()
    kws = set(list(kw_match_vc.keys()) + list(relevant_vc.keys()) + list(irrelevant_match_vc.keys()))
    out_data = {kw:{} for kw in kws}
    for kw in kws:
        if kw in kw_match_vc:
            out_data[kw]['kw_match'] = kw_match_vc[kw]
        else:
            out_data[kw]['kw_match'] = 0
        if kw in relevant_vc:
            out_data[kw]['relevant_count'] = relevant_vc[kw]
        else:
            out_data[kw]['relevant_count'] = 0
    out = pd.DataFrame.from_dict(out_data).T.reset_index().rename(columns={'index':col})
    out['relevant_perc'] = round((out.relevant_count) / (out.kw_match), 3)
    return out


def make_final_logs(text_df, sen_df, true_df, train_test_df, merged):
    print('I/O id summary')
    print('=======================================================================')
    print(pretty_str('all kw_match articles in raw data:', True))      # asserted by check_asserts()
    print(pretty_str('all matchedsentences in kw_match data:', True))
    print(pretty_str('unique articles:', len(text_df.id.unique())))
    print(pretty_str('unique articles w/ kw match:', len(sen_df.article_id.unique())))
    print(pretty_str('unique matched sentences:', len(sen_df.id.unique())))
    print(pretty_str('unique matched sentences relevant:', len(true_df.matchedsentence_id.unique())))
    print(pretty_str('unique matched officers relevant:', len(true_df.id.unique())))
    print(pretty_str('unique articles in train_test:', len(train_test_df.article_id.unique()), newline=True))
    return 1

In [4]:
# NEED TO OUTPUT:
# 1. article_id (given in data)
# 2. article text (given in data)
# 3. relevant (if article in true_df)
# 4. test (if article is reserved for testing model)     Per TS: 500 train, 100 test for initial train
# (may add cols like author or title)

# CONSIDERING
# correct_kw_match? does article_id conflict also occur with matchedsentence_id?

In [5]:
pd.set_option('mode.chained_assignment', 'raise')
# __main__

# newsarticle: initial dataset
#    - has all the data related to the article as it was pulled into feed
# matchedsentence: initial keyword filter
#    - has all the data related to every article with at least one sentence matching a keyword
# matchedsentence_officers: manual filter (Rajiv)
#    - has select columns linking identified officer badges and articles confirmed relevant by Rajiv
# NOTE: If an article is not in the manual filter set, it is not relevant
news_text_f = '../input/news_articles_newsarticle.csv.gz'
news_included_f = '../input/news_articles_matchedsentence.csv.gz'
news_true_f = '../input/news_articles_matchedsentence_officers.csv.gz'

text_df = open_gz(news_text_f)
sen_df = open_gz(news_included_f)
true_df = open_gz(news_true_f)
check_asserts(text_df, sen_df, true_df)

less_text, less_sen, less_true = prep_dfs(text_df, sen_df, true_df)
merged = merge_dfs(less_text, less_sen, less_true)
# report lost columns
print()
print('columns ignored by import')
print('=======================================================================')
all_cols = set(text_df.columns.tolist() + sen_df.columns.tolist() + true_df.columns.tolist())
kept = set(merged.columns)
not_kept = all_cols.difference(kept)
not_kept.remove('id')
print(str(not_kept)+'\n')

# make sure every article_id has a corresponding 'relevant' value
all_ids = set(merged.article_id.unique())
relevant_articles = set(merged.loc[(merged.relevant == 1)].article_id.unique())
irrelevant_articles = set(merged.loc[(merged.relevant == 0)].article_id.unique())
rel_vals = relevant_articles.union(irrelevant_articles)
assert all_ids.difference(rel_vals) == set()
overlap = relevant_articles.intersection(irrelevant_articles)
print('relevant && irrelevant articles check')
print('=======================================================================')
print(pretty_str('unique relevant articles:', len(relevant_articles)))
print(pretty_str('unique irrelevant articles:', len(irrelevant_articles)))
# check for conflicting 'relevant' values, correct if present
print(pretty_str('relevant and irrelevant disjoint:', overlap == set()))
if overlap != set():
    print(pretty_str('size of overlap:', len(overlap)))
    temp = merged
    merged = correct_relevant(temp)
    print(pretty_str('amended relevant column:', True, newline=True))

# Per TS, starting train/test size should be roughly 500/100
# ASSUMES initial balance of 50/50 positive/negative
# proceed with generating training data
merged = make_train_test_cols(merged, pos_rate=0.5)
train_test_df = make_train_test_df(merged)
print('train, test summary')
print('=======================================================================')
train = train_test_df.loc[train_test_df.test == 0, ['article_id', 'content', 'relevant']]
test = train_test_df.loc[train_test_df.test == 1, ['article_id', 'content', 'relevant']]
train_n = train.article_id.count()
test_n = test.article_id.count()
assert train_n + test_n == train_test_df.shape[0]
train_pos = train.loc[train.relevant == 1].article_id.count()
test_pos = test.loc[test.relevant == 1].article_id.count()
train_neg = train_n - train_pos
test_neg = test_n - test_pos
print(f'{train_n} datapoints available for training with balance: {train_pos}/{train_neg} (pos/neg)')
print(f'{test_n} datapoints available for testing with balance: {test_pos}/{test_neg} (pos/neg)\n')
print('train info')
print('=======================================================================')
train.info()
print('\ntest info')
print('=======================================================================')
test.info()
print()

# writing a subset of merged to use as pre-training data
temp = merged.loc[:, ['article_id', 'title', 'content']].drop_duplicates(subset='article_id')
assert len(temp.article_id.unique()) == temp.shape[0]
news = temp.sample(temp.shape[0]).reset_index(drop=True)

assert make_final_logs(text_df, sen_df, true_df, train_test_df, merged)

# generate keyword report (source_id, author also helpful reports)
kw_report = make_report(merged, 'extracted_keywords')
print('keyword report')
print('=======================================================================')
print('{:50}{:15}{:10}'.format('extracted_keywords', 'relevant_perc', 'kw_match'))
sorted_kw_report = kw_report[['extracted_keywords', 'relevant_perc', 'kw_match']].sort_values(by='relevant_perc', ascending=False)
for tup in sorted_kw_report.itertuples():
    print(pretty_str(tup.extracted_keywords+':', tup.relevant_perc, b=tup.kw_match))


columns ignored by import
{'created_at', 'link', 'guid', 'is_processed', 'updated_at', 'url', 'published_date'}

relevant && irrelevant articles check
unique relevant articles:                         316
unique irrelevant articles:                       29616
relevant and irrelevant disjoint:                 False
size of overlap:                                  225
amended relevant column:                          True

train, test summary
487 datapoints available for training with balance: 249/238 (pos/neg)
127 datapoints available for testing with balance: 67/60 (pos/neg)

train info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 487 entries, 25391 to 36107
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article_id  487 non-null    int64 
 1   content     487 non-null    object
 2   relevant    487 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 15.2+ KB

test info
<class 'pandas.core.frame.DataF

In [6]:
# Estimated true pos_rate
kw_match_n = merged.loc[merged.kw_match==1].article_id.count()
relevant_n = merged.loc[merged.relevant==1].article_id.count()
round(relevant_n/kw_match_n, 2)

0.15

In [7]:
# Are there any articles that contain a matched sentence but kw_match isn't True?
match = set(merged.loc[merged.kw_match == 1].article_id.unique())
no_match = set(merged.loc[merged.kw_match == 0].article_id.unique())
match.intersection(no_match)

set()

In [8]:
# outgoing train_test asserts:
#    1. No article_id or content is missing from either train or test sets
#    2. No article_id or content appears in both train AND test sets
#    3. Every article_id has a relevant, test value
#    - Could assert pos_rate with margin of error +- 0.1

# outgoing rule 1
assert train_test_df.loc[train_test_df.article_id.isnull()].shape == (0,4)
assert train_test_df.loc[train_test_df.content.isnull()].shape == (0,4)

# outgoing rule 2
train_articles = set(train_test_df.loc[train_test_df.test == 0].article_id.unique())
test_articles = set(train_test_df.loc[train_test_df.test == 1].article_id.unique())
assert train_articles.isdisjoint(test_articles)
train_contents = set(train_test_df.loc[train_test_df.test == 0].content.unique())
test_contents = set(train_test_df.loc[train_test_df.test == 1].content.unique())
print(train_contents.isdisjoint(test_contents))

# caveat to rule 2: duplicate 
article_ids = set(train_test_df.article_id.unique())
contents = set(train_test_df.content.unique())
print(train_test_df.shape)
assert len(article_ids) == train_test_df.shape[0]
print(len(contents))

# outgoing rule 3
assert train_test_df.loc[train_test_df.relevant.isnull()].shape == (0,4)
assert train_test_df.loc[train_test_df.test.isnull()].shape == (0,4)

False
(614, 4)
609


In [9]:
# Rule 1 violation: Why is unique content < number of rows
dup_content = train_test_df.loc[train_test_df.duplicated(subset='content')].content.values.tolist()
if dup_content != []:
    dup_content_df = train_test_df.loc[train_test_df.content.isin(dup_content)]
    print('duplicated content shape:', dup_content_df.shape)
    dup_content_ids = dup_content_df.article_id.unique().tolist()
    print('article_ids in train_test_df implicated by dup_content:', len(dup_content_ids))
    all_dup_content_ids = merged.loc[merged.article_id.isin(dup_content_ids)].article_id.unique().tolist()
    print('article_ids in merged implicated by dup_content:', len(all_dup_content_ids))

duplicated content shape: (10, 4)
article_ids in train_test_df implicated by dup_content: 10
article_ids in merged implicated by dup_content: 10


In [11]:
# Rule 1 violation: Why is unique content < number of rows
dup_content = merged.loc[merged.duplicated(subset='content')].content.values.tolist()
if dup_content != []:
    dup_content_df = merged.loc[merged.content.isin(dup_content)]
    print('duplicated content shape:', dup_content_df.shape)
    dup_content_ids = dup_content_df.article_id.unique().tolist()
    print('article_ids in merged implicated by dup_content:', len(dup_content_ids))

duplicated content shape: (10397, 13)
article_ids in merged implicated by dup_content: 3994


In [12]:
# Rule 1 violation: Why is unique content < number of rows
dup_title = merged.loc[merged.duplicated(subset='title')].title.values.tolist()
if dup_title != []:
    dup_title_df = merged.loc[merged.title.isin(dup_title)]
    print('duplicated title shape:', dup_title_df.shape)
    dup_title_ids = dup_title_df.article_id.unique().tolist()
    print('article_ids in merged implicated by dup_title:', len(dup_title_ids))
    dup_title_ids = dup_title_df.loc[dup_title_df.content].article_id.unique().tolist()

duplicated title shape: (11036, 13)
article_ids in merged implicated by dup_title: 4633


KeyError: "None of [Index([                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [14]:
# Are matchedsentence_ids unique by text, or by text in unique article?
dup_text = merged.loc[merged.duplicated(subset='text')].text.unique().tolist()
dup_text_df = merged.loc[merged.text.isin(dup_text)]

dup_text_ids = {}
for tup in dup_text_df.itertuples():
    this_art_id = tup.article_id
    this_mat_id = tup.matchedsentence_id
    if str(tup.text) != 'nan':
        this_art_id = tup.article_id
        this_mat_id = tup.matchedsentence_id
        if tup.text not in dup_text_ids:
            dup_text_ids[tup.text] = {'article_id': [this_art_id], 'matchedsentence_id': [this_mat_id]}
        else:
            if this_art_id not in dup_text_ids[tup.text]['article_id']:
                dup_text_ids[tup.text]['article_id'].append(this_art_id)
            if this_mat_id not in dup_text_ids[tup.text]['matchedsentence_id']:
                dup_text_ids[tup.text]['matchedsentence_id'].append(this_mat_id)

multiple_art = 0
for text, id_dict in dup_text_ids.items():
    if (len(id_dict['article_id']) > 1):
        multiple_art += 1
        # suspect sentences found in distinct articles are processed as distinct sentences
        # matchedsentence_id only refers to uniqueness within article, not in database
        assert len(id_dict['article_id']) == len(id_dict['matchedsentence_id'])

print('duplicate text shape:\t\t\t', dup_text_df.shape)
print('unique text samples duplicated:\t\t', len(dup_text_ids))
print('articles affected by duplicate text:\t', multiple_art)

duplicate text shape:			 (26765, 13)
unique text samples duplicated:		 580
articles affected by duplicate text:	 410


In [15]:
merged.loc[merged.extracted_keywords == "{'officer', 'terminated', 'police'}"]

,article_id,matchedsentence_id,source_id,author,title,text,content,officer_id,extracted_keywords,kw_match,relevant,train,test


In [16]:
merged.loc[merged.extracted_keywords == "{'terminated', 'police'}"]

,article_id,matchedsentence_id,source_id,author,title,text,content,officer_id,extracted_keywords,kw_match,relevant,train,test
28243,21632,9738.0,2,Davide Mamone,"Wayne Griffin, fired interim Lafayette police ...","Wayne Griffin, who was terminated amid sexual ...","Former Lafayette Police Sgt. Wayne Griffin, wh...",73862.0,"{'terminated', 'police'}",1,1,1,0
28244,21632,9738.0,2,Davide Mamone,"Wayne Griffin, fired interim Lafayette police ...","Wayne Griffin, who was terminated amid sexual ...","Former Lafayette Police Sgt. Wayne Griffin, wh...",98862.0,"{'terminated', 'police'}",1,1,1,0
29080,19097,8362.0,2,Davide Mamone,"Amid sexual allegations, Wayne Griffin fired f...",The former interim Lafayette Police Chief Wayn...,The former interim Lafayette Police Chief Wayn...,73862.0,"{'terminated', 'police'}",1,1,1,1
29081,19097,8362.0,2,Davide Mamone,"Amid sexual allegations, Wayne Griffin fired f...",The former interim Lafayette Police Chief Wayn...,The former interim Lafayette Police Chief Wayn...,98862.0,"{'terminated', 'police'}",1,1,1,1


In [17]:
relevant = merged.loc[merged.relevant == 1].sample(len(relevant_articles))
for tup in relevant.itertuples():
    print(tup.article_id)
    print(tup.title)
    print()

6876
Lafayette man killed in Thanksgiving night crash in Iberia Parish

8627
Psychologist: Shreveport man accused in postal worker's death competent to stand trial

29704
Man suing former LPD officer for civil rights violations represented by former police chief

10155
Susan Hutson elected Orleans Parish Sheriff

649
1 dead, 1 injured in West Azalea Avenue shooting, Baker police say

6419
Vermilion Parish Sheriff's Deputies Attempt to Capture Federal Fugitive; Ends with Fugitive Taking His Owns Life

2613
Former police officer convicted of molesting child under 13 in Acadia Parish

29874
BRPD discipline battle: Officer at center of feud with police chief wins civil service board seat

12526
ACLU pursues complaints of police misconduct statewide

22993
NOPD releases video of Mid-City officer-involved shooting

4695
Lafayette man arrested in connection to shots fired in Abbeville

9115
State Police discuss policy changes in new Senate Committee meeting

5668
New York man accused of killi

In [18]:
merged.loc[(merged.article_id == 4603)].content.unique()

array(['SHREVEPORT, La. -- Every week or so, an assistant U.S. Attorney in Shreveport meets with an agent from the Federal Bureau of Alcohol, Tobacco, Firearms and Explosive to go over illegal gun possession arrests made by local law enforcement agencies. They are looking for cases to take. It’s part of an effort across the country to respond to growing pressures to reduce the rise in gun violence. Federal prosecutors are taking more lower-level, run-of-the-mill gun cases that used to be the bailiwick of local prosecutors. Much of the time, defendants will face a higher probability of longer prison terms. Prosecutors are identifying cases with aggravating factors such as defendants with gang affiliations, a history of violence and involvement in drug trafficking. KTBS News reviewed indictments returned against eight men this summer and fall in Shreveport. Two of the defendants were suspects in murders; authorities allege all are members of gangs; and half have extensive criminal record

In [21]:
poss = [5668, 4151, 1710, 4841, 649]
like = [5266, 2327, 9541, 8325, 1536, 2978, 9404, 997, 9423]

merged.loc[merged.article_id.isin(poss)].content.unique()

array(['\nGRETNA. La. (WGNO) — On Wednesday, October 20, the Gretna Police Department made a traffic stop that resulted in the arrest of a man.\nGretna police officers, Jose DelosAngeles and Kaleab Magyar made a traffic stop for Joseph Adams, the driver of a Ford F-150.\nAccording to police, Adams was extremely nervous.\n\t\tPolice officer saves man who was struck by a train in Slidell\t\nThe behavior made police suspicious and believed that this was more than just a traffic stop.\nAfter officers conducted a registration check they learned the truck was involved in connection with a homicide that happened in New York. The vehicle had a hold on it for NYPD.\nAdams was detained immediately after. Police also found drugs and a loaded firearm in a search.\nAdams had a criminal history of first-degree rape and illegal carry of a firearm.\n\t\tNew Orleans Heart Walk reunites the  community for better health\t\nHe was placed under arrest and booked into the Jefferson Parish Correctional Cente

In [1]:
# True if police shooting/misconduct/trials/terminations in content
# 'possibly False' if content describes an arrest/traffic stop but no misconduct/trials/etc
# 'likely False' if content is PSA, bureaucracy; mentions police, but not police actions/accusations
# Arrest summaries (ie. 8306) could be parsed with regex to filter for officer arrest
manual_audit = {
    2613: True,
    5668: 'possibly False',
    5266: 'likely False',
    4151: 'possibly False',
    10632: True,
    7688: 'possibly False',   # victim died from injuries after being found by police
    1434: True,
    7330: True,
    27057: True,
    21559: True,
    9790: 'possibly False',
    4582: 'possibly False',
    4985: 'possibly False',
    5910: 'possibly False',   # community expressed fear, police got video cameras, overtime
    2327: 'likely False',
    4003: True,    # participant in Capitol riots telling his story, tear gas, police shooting
    1091: 'possibly False',   # victim died from injuries after being found by police
    9541: 'likely False',     # details a new tech/lab police gained access to
    1199: True,
    6123: 'likely False',
    9021: True,
    1710: 'possibly False',
    1935: True,
    1937: True,
    8325: 'likely False',
    1780: True,
    1463: 'possibly False',   # school hired felon when background check from police took 21 days, police victim blame
    460: 'likely False',
    7323: 'likely False',     # details of Jessie Smollett case
    1502: True,
    8627: 'possibly False',
    29874: True,
    6209: True,
    1536: 'likely False',
    2978: 'likely False',
    1679: True,
    9413: True,
    1277: True,
    9005: True,
    20131: True,
    1116: True,
    7432: True,
    12526: True,
    4841: 'possibly False',
    5669: 'possibly False',   # pseudo-unique?
    9501: 'likely False',
    405: True,
    8718: 'possibly False',
    18610: True,   # pseudo-unique?
    3821: 'likely False',     # trials details but 
    2395: 'likely False',
    768: 'likely False',
    3845: True,
    8371: True,
    8306: 'possibly False',
    2902: True,
    13800: 'possibly False',
    1499: True,
    7712: 'possibly False',
    10250: 'possibly False',
    8159: 'likely False',
    1478: True,
    19097: True,
    20671: True,
    2459: True,
    6051: 'possibly False',     # describes a court order to seal a case where a man killed a cop
    10275: 'likely False',
    3267: 'likely False',
    674: 'likely False',
    1840: 'likely False',
    7663: 'possibly False',
    2369: True,
    2201: 'likely False',
    3332: 'possibly False',
    10273: 'likely False',
    1690: True,
    8045: True,
    1502: True,
    9449: 'likely False',
    10255: 'likely False',
    4690: 'possibly False',   # community expressed fear, police got video cameras, overtime
    649: 'possibly False',   # victim died from injuries after being found by police
    10488: True,     # judge used racial slurs when her home was burglarized, was sedated atm 
    9600: True,
    9423: 'likely False',
    2946: 'likely False',
    7360: 'possibly False',
    15599: True,
    4281: True,
    1096: True,
    11073: True,
    4538: 'likely False',
    1345: 'likely False',
    21632: True,
    30695: True,
    21285: True,
    5712: 'likely False',    # misconduct but on the part of athletes and coaches, RE: title xi
    8060: True,
    10166: True,
    30616: True,
    7896: 'likely False',
    780: 'possibly False',   # victim died from injuries after being found by police
    742: True,
    9897: True,
    997: 'likely False',
    9404: 'likely False',    # police showing off a training exercise: deadly force at nighttime
    4603: True,     # 'if you have someone you really want to put away'
}

In [6]:
print('inspected:\t', len(manual_audit))
print('True positive:\t', len([1 for key,val in manual_audit.items() if val == True]))
print('possibly False:\t', len([1 for key,val in manual_audit.items() if val == 'possibly False']))
print('likely False:\t', len([1 for key,val in manual_audit.items() if val == 'likely False']))

inspected:	 106
True positive:	 51
possibly False:	 25
likely False:	 30


# Reviewing output data

In [6]:
train_test_df

,article_id,content,relevant,test
25391,31316,A prosecutor is denying accusations that detec...,1,0
25403,31266,(The Center Square) — Motorists on the Atchafa...,0,0
25404,31264,"Shreveport, La -- A Shreveport man who police ...",0,0
25410,31255,A 25-year-old man has been identified as the p...,0,0
25473,31151,\nNEW ORLEANS (WGNO) — On Monday morning the U...,0,0
...,...,...,...,...
36030,460,"Despite a slow down because of Hurricane Ida, ...",1,0
36040,413,The Innocence Project New Orleans has filed ba...,0,0
36045,405,A Louisiana State Police trooper who initiated...,1,1
36097,391,The number of COVID-19 cases among people inca...,0,0


In [7]:
merged

,article_id,matchedsentence_id,source_id,author,text,content,officer_id,extracted_keywords,kw_match,relevant,train,test
0,31383,NaN,4,Keisha Swafford,NaN,Living with diabetes is a lifetime challenge f...,NaN,NaN,0,0,0,0
1,31381,NaN,2,Charles Salzer,NaN,As the Live Oak softball team begins postseaso...,NaN,NaN,0,0,0,0
2,31380,NaN,2,Community News Report,NaN,North Oaks Sports Medicine certified athletic ...,NaN,NaN,0,0,0,0
3,31378,NaN,2,Community News Report,NaN,The Great American Cleanup Love the Boot event...,NaN,NaN,0,0,0,0
4,31377,NaN,2,Community News Report,NaN,Gordon McKernan Injury Attorneys has launched ...,NaN,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
36105,373,9.0,1,Marta Jewson,Chief Operations Officer Tiffany Delcour estim...,"At a press conference Wednesday, NOLA Public S...",NaN,{'officer'},1,0,1,0
36106,366,3.0,1,Carly Berlin,At a Monday press conference with New Orlean...,"UPDATE: After this story was published, FEMA e...",NaN,{'officer'},1,0,0,0
36107,365,2.0,1,Marta Jewson,"Before students return to any campus, NOLA P...",Outside Frederick Douglass High School Thursda...,NaN,{'officer'},1,0,0,0
36108,362,1.0,1,Marta Jewson,"Damage report As of Tuesday, roughly half of t...","About 250,000 Louisiana students remain out of...",NaN,{'officer'},1,0,0,0


## Reports
- `extracted_keywords` in train/test/rem
- sources in train/test/rem
- authors in train/test/rem

In [8]:
kw_report.sort_values(by='relevant_perc', ascending=False)

,extracted_keywords,kw_match,relevant_count,relevant_perc
6,"{'police', 'terminated'}",4,4,0.500
0,"{'terminated', 'officer'}",4,3,0.429
11,"{'nopd', 'police', 'officer'}",5,2,0.286
5,"{'police', 'officer'}",889,191,0.177
8,"{'nopd', 'officer'}",70,12,0.146
9,{'terminated'},91,14,0.133
2,{'officer'},2764,422,0.132
4,{'police'},6350,938,0.129
10,"{'police', 'nopd', 'officer'}",7,1,0.125
7,"{'nopd', 'police'}",20,2,0.091


In [31]:
print(kw_report[['extracted_keywords', 'relevant_perc']].sort_values(by='relevant_perc', ascending=False))

                     extracted_keywords  relevant_perc
6              {'police', 'terminated'}          0.500
0             {'terminated', 'officer'}          0.429
11        {'nopd', 'police', 'officer'}          0.286
5                 {'police', 'officer'}          0.177
8                   {'nopd', 'officer'}          0.146
9                        {'terminated'}          0.133
2                           {'officer'}          0.132
4                            {'police'}          0.129
10        {'police', 'nopd', 'officer'}          0.125
7                    {'nopd', 'police'}          0.091
12                             {'nopd'}          0.079
1   {'police', 'terminated', 'officer'}          0.000
3                    {'police', 'nopd'}          0.000


In [9]:
src_report.sort_values(by='relevant_count', ascending=False)

,source_id,kw_match,relevant_count,relevant_perc
1,2,3345,790,0.191
18,19,1497,144,0.088
15,16,831,107,0.114
0,1,255,75,0.227
16,17,154,68,0.306
23,25,102,67,0.396
28,30,1204,53,0.042
6,7,528,43,0.075
30,32,303,42,0.122
13,14,304,32,0.095


In [10]:
author_report.sort_values(by='relevant_count', ascending=False)

,author,kw_match,relevant_count,relevant_perc
266,Katie Gagliano,226,103,0.313
4,Claire Taylor,152,100,0.397
522,James Finn,237,81,0.255
235,Staff Report,437,68,0.135
179,Joe Gyan Jr.,167,59,0.261
...,...,...,...,...
239,Ron Faucheux,2,0,0.000
238,Bess Casserleigh,2,0,0.000
237,WBRZ News,4,0,0.000
234,Special to The Town Talk,2,0,0.000


# Reviewing input data

### `text_df`

In [11]:
pretty_print('full text shape:', text_df.shape)
print('full text cols:\n', list(text_df.columns))

full text shape:                                  (29707, 12)
full text cols:
 ['created_at', 'link', 'guid', 'source_id', 'updated_at', 'content', 'published_date', 'id', 'title', 'is_processed', 'author', 'url']


In [12]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29707 entries, 0 to 29706
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   created_at      29707 non-null  object
 1   link            29707 non-null  object
 2   guid            29707 non-null  object
 3   source_id       29707 non-null  int64 
 4   updated_at      29707 non-null  object
 5   content         29254 non-null  object
 6   published_date  29707 non-null  object
 7   id              29707 non-null  int64 
 8   title           29707 non-null  object
 9   is_processed    29707 non-null  bool  
 10  author          27270 non-null  object
 11  url             29707 non-null  object
dtypes: bool(1), int64(2), object(9)
memory usage: 2.5+ MB


In [13]:
get_unique_report(text_df)

             distinct value count by col
created_at                                        29707
link                                              29707
guid                                              29698
source_id                                         34
updated_at                                        29707
content                                           28581
published_date                                    529
id                                                29707
title                                             28392
is_processed                                      1
author                                            2200
url                                               29707


### `sen_df`

In [14]:
pretty_print('matched sen shape:', sen_df.shape)
print('matched sen cols:\n', list(sen_df.columns))

matched sen shape:                                (10470, 7)
matched sen cols:
 ['id', 'created_at', 'updated_at', 'article_id', 'extracted_keywords', 'text', 'title']


In [15]:
sen_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10470 entries, 0 to 10469
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  10470 non-null  int64 
 1   created_at          10470 non-null  object
 2   updated_at          10470 non-null  object
 3   article_id          10470 non-null  int64 
 4   extracted_keywords  10470 non-null  object
 5   text                10470 non-null  object
 6   title               10470 non-null  object
dtypes: int64(2), object(5)
memory usage: 572.7+ KB


In [16]:
get_unique_report(sen_df)

             distinct value count by col
id                                                10470
created_at                                        10470
updated_at                                        10470
article_id                                        4323
extracted_keywords                                60
text                                              9925
title                                             4179


### `true_df`

In [17]:
pretty_print('matched true shape:', true_df.shape)
print('matched true cols:\n', list(true_df.columns))

matched true shape:                               (735, 3)
matched true cols:
 ['id', 'matchedsentence_id', 'officer_id']


In [18]:
true_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   id                  735 non-null    int64
 1   matchedsentence_id  735 non-null    int64
 2   officer_id          735 non-null    int64
dtypes: int64(3)
memory usage: 17.4 KB


In [19]:
get_unique_report(true_df)

             distinct value count by col
id                                                355
matchedsentence_id                                479
officer_id                                        355
